# Explore the AUG workflow and single flux tube UQ data sets

- this uses 
  - uq/data/AUG_mix-lim_gem_inoutput.txt for the WF data
    - with limits on Te, dTe, Ti and dTi
  - uq/data/gem_uq_inoutput.csv for the single flux tube UQ data

## A fairly standard set of imports

In [1]:
%matplotlib inline
#%matplotlib notebook
import os
import matplotlib
if not os.getenv("DISPLAY"): matplotlib.use('Agg')
import matplotlib.pylab as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')
import numpy as np
import pandas as pd
pd.options.display.max_rows = 100
from utility import _ewma, plot_smoothed, plot_unsmoothed

## Read and examine the workflow data

In [2]:
# read the data from the AUG workflow runs
AUG_gem1 = pd.read_table('../../data/AUG_gem_inoutput.txt', delimiter='  *', engine='python')
AUG_gem2 = pd.read_table('../../data/AUG_mix-lim_gem_inoutput.txt', delimiter='  *', engine='python') 

In [3]:
AUG_gem1.describe().T

,count,mean,std,min,25%,50%,75%,max
time,7900.0,2.838585e+00,7.433801e-01,2.005000e+00,2.149560e+00,2.617608e+00,3.406262e+00,4.347568e+00
Te-ft1,7900.0,2.664787e+03,2.597216e+02,2.166346e+03,2.408818e+03,2.814114e+03,2.890374e+03,2.962908e+03
Te-ft2,7900.0,2.289331e+03,2.219132e+02,1.885349e+03,2.071053e+03,2.421143e+03,2.468700e+03,2.569747e+03
Te-ft3,7900.0,1.834605e+03,1.299057e+02,1.593019e+03,1.704682e+03,1.897137e+03,1.936392e+03,2.040789e+03
Te-ft4,7900.0,1.367150e+03,2.994889e+01,1.291171e+03,1.345756e+03,1.364429e+03,1.386958e+03,1.482816e+03
Te-ft5,7900.0,1.065808e+03,3.259132e+01,9.940712e+02,1.041249e+03,1.065619e+03,1.087060e+03,1.207042e+03
Te-ft6,7900.0,8.389278e+02,3.259963e+01,7.371233e+02,8.185224e+02,8.417796e+02,8.596861e+02,9.937032e+02
Te-ft7,7900.0,6.128547e+02,2.512077e+01,5.482888e+02,5.990524e+02,6.147991e+02,6.276373e+02,7.841894e+02
Te-ft8,7900.0,3.388987e+02,1.842898e+01,2.994911e+02,3.298905e+02,3.377372e+02,3.462935e+02,5.911335e+02
dTe-ft1,7900.0,-2.492175e+03,3.792470e+02,-4.643766e+03,-2.627039e+03,-2.406545e+03,-2.284707e+03,-1.463287e+03


In [4]:
AUG_gem2.describe().T

,count,mean,std,min,25%,50%,75%,max
time,47400.0,2.243139e+00,8.326989e-01,9.950002e-01,1.556172e+00,2.178609e+00,2.741358e+00,4.256661e+00
Te-ft1,47400.0,2.506065e+03,1.078970e+02,2.166346e+03,2.435510e+03,2.486968e+03,2.539434e+03,2.743381e+03
Te-ft2,47400.0,2.105301e+03,1.106564e+02,1.918080e+03,2.041692e+03,2.056203e+03,2.117686e+03,2.353236e+03
Te-ft3,47400.0,1.599099e+03,4.639509e+01,1.521299e+03,1.558100e+03,1.587158e+03,1.636915e+03,1.716907e+03
Te-ft4,47400.0,1.247846e+03,4.830985e+01,1.178360e+03,1.212111e+03,1.227567e+03,1.281057e+03,1.434139e+03
Te-ft5,47400.0,1.004322e+03,4.830321e+01,9.343535e+02,9.655175e+02,9.860457e+02,1.039702e+03,1.207042e+03
Te-ft6,47400.0,7.977915e+02,3.469572e+01,7.227375e+02,7.696538e+02,7.903375e+02,8.231823e+02,9.921931e+02
Te-ft7,47400.0,6.010339e+02,1.671097e+01,5.490308e+02,5.897889e+02,6.005970e+02,6.110884e+02,7.841957e+02
Te-ft8,47400.0,3.556167e+02,1.195392e+01,3.211371e+02,3.507044e+02,3.575621e+02,3.619766e+02,5.912203e+02
dTe-ft1,47400.0,-1.900973e+03,4.483971e+02,-3.197475e+03,-2.336070e+03,-1.703934e+03,-1.568622e+03,-1.218317e+03


In [5]:
AUG_tmp = pd.read_table('../../data/AUG_mix-lim_flux_area.txt', delimiter='  *', engine='python')
AUG_tmp.describe().T



,count,mean,std,min,25%,50%,75%,max
time,47400.0,2.243139e+00,0.832699,9.950002e-01,1.556172e+00,2.178609e+00,2.741358e+00,4.256661e+00
Te-source-int-ft1,47400.0,1.130674e+05,1431.974980,1.093967e+05,1.122041e+05,1.126609e+05,1.140166e+05,1.213880e+05
Te-source-int-ft2,47400.0,4.666860e+05,6334.990210,4.562429e+05,4.625731e+05,4.648170e+05,4.704037e+05,5.174659e+05
Te-source-int-ft3,47400.0,8.372903e+05,10508.850868,8.229965e+05,8.313692e+05,8.352535e+05,8.415945e+05,9.413594e+05
Te-source-int-ft4,47400.0,1.165744e+06,14140.756365,1.148406e+06,1.158686e+06,1.162991e+06,1.170477e+06,1.325870e+06
Te-source-int-ft5,47400.0,1.435996e+06,18117.624018,1.413970e+06,1.426763e+06,1.432014e+06,1.443204e+06,1.654474e+06
Te-source-int-ft6,47400.0,1.646224e+06,23480.262134,1.614888e+06,1.630625e+06,1.642724e+06,1.658681e+06,1.920913e+06
Te-source-int-ft7,47400.0,1.804165e+06,31767.818436,1.752776e+06,1.778345e+06,1.803103e+06,1.824816e+06,2.138927e+06
Te-source-int-ft8,47400.0,1.934790e+06,42087.650299,1.857696e+06,1.899112e+06,1.934699e+06,1.965612e+06,2.338017e+06
Ti-source-int-ft1,47400.0,1.373439e+05,1466.608239,1.315166e+05,1.363946e+05,1.369136e+05,1.377631e+05,1.412150e+05


### Set up some column selections

In [6]:
active_t = []
active_t.append(['time', 'Te-ft1', 'dTe-ft1', 'Ti-ft1', 'dTi-ft1', 'flux-Te-ft1', 'flux-Ti-ft1'])
active_t.append(['time', 'Te-ft2', 'dTe-ft2', 'Ti-ft2', 'dTi-ft2', 'flux-Te-ft2', 'flux-Ti-ft2'])
active_t.append(['time', 'Te-ft3', 'dTe-ft3', 'Ti-ft3', 'dTi-ft3', 'flux-Te-ft3', 'flux-Ti-ft3'])
active_t.append(['time', 'Te-ft4', 'dTe-ft4', 'Ti-ft4', 'dTi-ft4', 'flux-Te-ft4', 'flux-Ti-ft4'])
active_t.append(['time', 'Te-ft5', 'dTe-ft5', 'Ti-ft5', 'dTi-ft5', 'flux-Te-ft5', 'flux-Ti-ft5'])
active_t.append(['time', 'Te-ft6', 'dTe-ft6', 'Ti-ft6', 'dTi-ft6', 'flux-Te-ft6', 'flux-Ti-ft6'])
active_t.append(['time', 'Te-ft7', 'dTe-ft7', 'Ti-ft7', 'dTi-ft7', 'flux-Te-ft7', 'flux-Ti-ft7'])
active_t.append(['time', 'Te-ft8', 'dTe-ft8', 'Ti-ft8', 'dTi-ft8', 'flux-Te-ft8', 'flux-Ti-ft8'])

## Read and examine the UQ data

In [7]:
# read the data from the AUG single flux tube UQ runs
G = pd.read_csv('../../data/gem_uq_inoutput.csv')
G.describe().T

,count,mean,std,min,25%,50%,75%,max
te_value,625.0,1118.873710,447.907954,479.555982,815.518812,1118.873710,1422.228609,1758.191438
ti_value,625.0,1055.702039,422.619046,452.480225,769.474575,1055.702039,1341.929503,1658.923853
te_ddrho,625.0,-3138.035033,1256.219391,-4931.089432,-3988.835522,-3138.035033,-2287.234544,-1344.980635
ti_ddrho,625.0,-2420.640014,969.031541,-3803.779201,-3076.936609,-2420.640014,-1764.343419,-1037.500827
te_transp_flux,625.0,6972.636645,11490.477446,220.214756,1525.539253,3760.020921,8067.792453,160763.775878
ti_transp_flux,625.0,6376.773923,16145.588239,198.048746,1234.113504,1984.548333,3304.507114,123668.654262


### Set up some column selections

In [ ]:
# setup some column groups 
Te_cols = ['Te-ft1', 'Te-ft2', 'Te-ft3', 'Te-ft4', 'Te-ft5', 'Te-ft6', 'Te-ft7', 'Te-ft8']
Ti_cols = ['Ti-ft1', 'Ti-ft2', 'Ti-ft3', 'Ti-ft4', 'Ti-ft5', 'Ti-ft6', 'Ti-ft7', 'Ti-ft8']
dTe_cols = ['dTe-ft1', 'dTe-ft2', 'dTe-ft3', 'dTe-ft4', 'dTe-ft5', 'dTe-ft6', 'dTe-ft7', 'dTe-ft8']
dTi_cols = ['dTi-ft1', 'dTi-ft2', 'dTi-ft3', 'dTi-ft4', 'dTi-ft5', 'dTi-ft6', 'dTi-ft7', 'dTi-ft8']
Te_flux_cols = ['flux-Te-ft1', 'flux-Te-ft2', 'flux-Te-ft3', 'flux-Te-ft4', 
                'flux-Te-ft5', 'flux-Te-ft6', 'flux-Te-ft7', 'flux-Te-ft8'] 
Ti_flux_cols = ['flux-Ti-ft1', 'flux-Ti-ft2', 'flux-Ti-ft3', 'flux-Ti-ft4', 
                'flux-Ti-ft5', 'flux-Ti-ft6', 'flux-Ti-ft7', 'flux-Ti-ft8']
ne_cols = ['ne-ft1', 'ne-ft2', 'ne-ft3', 'ne-ft4', 'ne-ft5', 'ne-ft6', 'ne-ft7', 'ne-ft8']
dne_cols = ['dne-ft1', 'dne-ft2', 'dne-ft3', 'dne-ft4', 'dne-ft5', 'dne-ft6', 'dne-ft7', 'dne-ft8']
ne_flux_cols = ['flux-ne-ft1', 'flux-ne-ft2', 'flux-ne-ft3', 'flux-ne-ft4', 
                'flux-ne-ft5', 'flux-ne-ft6', 'flux-ne-ft7', 'flux-ne-ft8']

In [ ]:
# setup some column groups 
dvdrho_cols = ['dvdrho1', 'dvdrho2', 'dvdrho3', 'dvdrho4', 'dvdrho5', 'dvdrho6', 'dvdrho7', 'dvdrho8']
surfaceA_cols = ['surfaceA1', 'surfaceA2', 'surfaceA3', 'surfaceA4', 'surfaceA5', 'surfaceA6', 'surfaceA7', 'surfaceA8']
Te_source_int_cols = ['Te-source-int-ft1', 'Te-source-int-ft2', 'Te-source-int-ft3', 'Te-source-int-ft4', 
                'Te-source-int-ft5', 'Te-source-int-ft6', 'Te-source-int-ft7', 'Te-source-int-ft8'] 
Ti_source_int_cols = ['Ti-source-int-ft1', 'Ti-source-int-ft2', 'Ti-source-int-ft3', 'Ti-source-int-ft4', 
                'Ti-source-int-ft5', 'Ti-source-int-ft6', 'Ti-source-int-ft7', 'Ti-source-int-ft8']
E_source_int_cols = ['E-source-int-ft1', 'E-source-int-ft2', 'E-source-int-ft3', 'E-source-int-ft4', 
                'E-source-int-ft5', 'E-source-int-ft6', 'E-source-int-ft7', 'E-source-int-ft8']
E_flux_cols = ['flux-E-ft1', 'flux-E-ft2', 'flux-E-ft3', 'flux-E-ft4', 
               'flux-E-ft5', 'flux-E-ft6', 'flux-E-ft7', 'flux-E-ft8']

In [ ]:
AUG_tmp[E_source_int_cols] = np.array(AUG_tmp[Te_source_int_cols]) + np.array(AUG_tmp[Ti_source_int_cols])
AUG_tmp[Te_flux_cols] = np.array(AUG_tmp[Te_source_int_cols]) / np.array(AUG_tmp[dvdrho_cols])
AUG_tmp[Ti_flux_cols] = np.array(AUG_tmp[Ti_source_int_cols]) / np.array(AUG_tmp[dvdrho_cols])
AUG_tmp[E_flux_cols] = np.array(AUG_tmp[E_source_int_cols]) / np.array(AUG_tmp[dvdrho_cols])
AUG_gem2[E_flux_cols] = np.array(AUG_gem2[Te_flux_cols]) + np.array(AUG_gem2[Ti_flux_cols])

In [ ]:
AUG_tmp[E_flux_cols].describe().T

In [ ]:
fig, ax = plt.subplots(4, 2, figsize=(24,24))
AUG_tmp.plot(x='time', y=dvdrho_cols, ax=ax[0,0])
AUG_tmp.plot(x='time', y=surfaceA_cols, ax=ax[0,1])
AUG_tmp.plot(x='time', y=Te_source_int_cols, ax=ax[1,0])
AUG_tmp.plot(x='time', y=Ti_source_int_cols, ax=ax[1,1])
AUG_tmp.plot(x='time', y=Te_flux_cols, ax=ax[2,0])
AUG_tmp.plot(x='time', y=Ti_flux_cols, ax=ax[2,1])
AUG_tmp.plot(x='time', y=E_source_int_cols, ax=ax[3,0])
AUG_tmp.plot(x='time', y=E_flux_cols, ax=ax[3,1]);

In [ ]:
import importlib, utility
importlib.reload(utility)
from utility import _ewma, plot_smoothed, plot_unsmoothed

## Flux tube electron and ion energy fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (1e2,1e6)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (1e2,1e6)}],
    logy=True)

### Smoothed electron and ion energy fluxes

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (1e2,0.7e5)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (1e2,1.2e5)}])

In [ ]:
plot_smoothed(
    [{"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": Te_flux_cols, "label": 'electron energy flux density [W m^{-2}]', "limit": (0e4,0.7e5)},
     {"cols": Ti_flux_cols, "label": 'ion energy flux density [W m^{-2}]', "limit": (0e4,1.2e5)}],
    logy=False)

## Flux tube Te & Ti

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": Te_cols, "label": 'electron temperature [eV]', "limit": (200,3500)},
     {"cols": Ti_cols, "label": 'ion temperature [eV]', "limit": (200,2500)}])

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": Te_cols, "label": 'electron temperature [eV]', "limit": (200,3500)},
     {"cols": Ti_cols, "label": 'ion temperature [eV]', "limit": (200,2500)}])

## Flux tube dTe & dTi

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": dTe_cols, "label": 'electron temperature gradient [eV/m]', "limit": (-8000,0)},
     {"cols": dTi_cols, "label": 'ion temperature gradient [eV/m]', "limit": (-5000,0)}],
    logy=False)

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": dTe_cols, "label": 'electron temperature gradient [eV/m]', "limit": (-8000,0)},
     {"cols": dTi_cols, "label": 'ion temperature gradient [eV/m]', "limit": (-5000,0)}],
    logy=False)

## Flux tube ne & dne (not evolved so should be pretty much constant)

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": ne_cols, "label": 'electron density [m^{-3}]', "limit": (3.5e19,4.5e19)},
     {"cols": dne_cols, "label": 'electron density gradient [m^{-3}/m]', "limit": (-0.65e19,-0.45e19)}],
    logy=False)

## Flux tube ne fluxes

In [ ]:
plot_unsmoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-2e20,2e20)},
     {"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-1e20,1e20)}],
    logy=False)

In [ ]:
plot_smoothed(
    [{"data": AUG_gem1, "style": '-.'}, {"data": AUG_gem2, "style": ':'}],
    [{"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-2e20,2e20)},
     {"cols": ne_flux_cols, "label": 'electron particle flux [s^{-1}]', "limit": (-1e20,1e20)}],
    logy=False)

## Summed energy fluxes

In [ ]:
plot_smoothed(
    [{"data": AUG_gem2, "style": '-'}, {"data": AUG_tmp, "style": ':'}],
    [{"cols": E_flux_cols, "label": 'summed energy flux density [W m^{-2}]', "limit": (0e4,2.0e5)}],
    logy=False, figsize=(24,16))

In [ ]:
import utility, importlib
importlib.reload(utility)
from utility import _ewma, plot_smoothed, plot_unsmoothed

# The End